# 19대 대선 시각화  
아래 참조  
https://blog.naver.com/owo2466/221669798756

In [ ]:
import pandas as pd
import numpy as np
import platform
import matplotlib.pyplot as plt

%matplotlib inline

path = "c:/Windows/Fonts/malgun.ttf"
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')    

plt.rcParams['axes.unicode_minus'] = False

In [ ]:
from selenium import webdriver
import time

In [ ]:
#driver = webdriver.Chrome('../driver/chromedriver')
driver = webdriver.Chrome("C:/Users/user/Anaconda3/chromedriver_win32/chromedriver.exe")
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09")

In [ ]:
driver.find_element_by_id("electionType1").click()

In [ ]:
driver.find_element_by_id("electionName").send_keys("제19대")

In [ ]:
driver.find_element_by_id("electionCode").send_keys("대통령선거")

In [ ]:
#화면에서 시,도 코드를 얻어옴
sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")
sido_list = sido_list_raw.find_elements_by_tag_name("option")
sido_names_values = [option.text for option in sido_list]
sido_names_values = sido_names_values[2:]
sido_names_values

In [ ]:
import re

def get_num(tmp):
    return float(re.split('\(', tmp)[0].replace(',',''))

위 모듈을 import해서 wait.until 함수를 사용한다.
WebDriverWait 함수는 검색버튼이 클릭가능할 때 까지 기다리는 기능을 하고,
move_sido 함수는 광역시도 이름을 리스트에 전송하고 검색 버튼을 누르는 역할을 한다.

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

wait = WebDriverWait(driver, 10)

def move_sido(name):
    element = driver.find_element_by_id("cityCode")
    element.send_keys(name)
    make_xpath = """//*[@id="searchBtn"]"""
    wait.until(EC.element_to_be_clickable((By.XPATH,make_xpath)))
    driver.find_element_by_xpath(make_xpath).click()

In [ ]:
# append_data 함수는 빈 내용으로 미리 준비한 DataFrame에 append 명령으로 
# 읽은 데이터를 하나씩 추가하는 기능을 가지고 있다.

def append_data(df, sido_name, data):
    for each in df[0].values[1:]:
        data['광역시도'].append(sido_name)
        data['시군'].append(each[0])
        data['pop'].append(each[2])
        data['moon'].append(get_num(each[3])) # 괄호가 있는 데이터만 미리 만들어둔 함수를 사용해준다.
        data['hong'].append(get_num(each[4]))
        data['ahn'].append(get_num(each[5]))

In [ ]:
# 미리 변수를 하나 만들고

election_result_raw = {'광역시도' : [],
                       '시군' : [],
                       'pop' : [],
                       'moon' : [],
                       'hong' : [],    
                       'ahn' : [] }

In [ ]:
from bs4 import BeautifulSoup

for each_sido in sido_names_values:
    move_sido(each_sido)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    
    df = pd.read_html(str(table))
    
    append_data(df, each_sido, election_result_raw)

In [ ]:
# 데이터를 가겨와 DataFrame 형태로 만들어 준다.

election_result = pd.DataFrame(election_result_raw, 
                               columns=['광역시도', '시군', 'pop', 'moon','hong','ahn'])
election_result.head(10)

In [ ]:
election_result.to_csv('../testData/election_result.csv', encoding='utf-8', sep=',')

In [ ]:
df분석

In [ ]:
# 저장한 엑셀파일을 불러온다.
election_result = pd.read_csv('../testData/election_result.csv', encoding='utf-8', 
                              index_col=0)
election_result.head()

In [ ]:
sido_candi =  election_result['광역시도']

In [ ]:
#광역시만 이름정리
sido_candi = [name[:2] 
            if name[:2] in ['서울','부산','대구','광주','인천','대전','울산']
            else '' for name in sido_candi]

In [ ]:
# 두 글자의 구 이름은 그대로 보내고 3가지이름의 구 이름은 2자리수로 줄인다.
def cut_char_sigu(name):
    return name if len(name)==2 else name[:-1]

In [ ]:
# 광역시가 아닌데 행정구를 가지고 있는 시에 대해 정리를 해준다.

import re

sigun_candi = ['']*len(election_result)

for n in election_result.index:
    each = election_result['시군'][n]
    if each[:2] in ['수원', '성남','안양','안산','고양',
                            '용인','청주','천안','전주','포항','창원']:
        sigun_candi[n] = re.split('시', each)[0]+' '+ \
                                                        cut_char_sigu(re.split('시', each)[1])
    else:
        sigun_candi[n] = cut_char_sigu(each)
        
sigun_candi[:10]

In [ ]:
# 광역시이름이 저장된 변수 'sido_candi'와 시군구 이름이 저장된 변수 'sigun_candi'를 합친다.
# sido_candi 변수에 공란이 있으면 첫 글자가 띄어쓰기가 될 수 있기 때문에 정리를 해준다.

ID_candi = [sido_candi[n]+' '+sigun_candi[n] for n in range(0,len(sigun_candi))]

ID_candi = [name[1:] if name[0]==' ' else name for name in ID_candi]
ID_candi = [name[:2] if name[:2]=='세종' else name for name in ID_candi] # 세종시는 예외 처리한다.

ID_candi[100:113]

In [ ]:
# ID_candi 변수를 election_result에 합친다.

election_result['ID'] = ID_candi
election_result.head(10)

In [ ]:
# 세 후보의 득표수에서 투표자수를 나눠서 각각의 득표율을 계산한다.

election_result[['rate_moon','rate_hong','rate_ahn']] = \
            election_result[['moon','hong','ahn']].div(election_result['pop'],  axis=0)
election_result[['rate_moon','rate_hong','rate_ahn']] *= 100
election_result.head()


In [ ]:
# 문 후보가 높은 비율로 득표한 지역이다.

election_result.sort_values(['rate_moon'], ascending=[False]).head(10)

In [ ]:
draw_korea_raw = pd.read_excel('../testData/draw_korea_raw.xlsx')
draw_korea_raw.head(3)

In [ ]:
draw_korea_raw_starcked = pd.DataFrame(draw_korea_raw.stack())
draw_korea_raw_starcked.reset_index(inplace=True)
draw_korea_raw_starcked.rename(columns={'level_0':'y', 'level_1':'x', 0:'ID'}, inplace=True)
draw_korea_raw_starcked.head()

In [ ]:
draw_korea_raw_starcked.to_csv('../testData/draw_korea_raw.csv', encoding='utf-8', sep=',')

In [ ]:
# 지역별 좌표 정보를 가지고 있는 파일을 불러온다.

draw_korea = pd.read_csv('../testData/draw_korea_raw.csv', encoding='utf-8', index_col=0)
draw_korea.head()

draw_korea의 ID와 election_result의 ID가 일치해야 하기 때문에 서로의 차집합이 공집합인지 알아본다.

In [ ]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

In [ ]:
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

In [ ]:
election_result[election_result['ID'] == '고성']

In [ ]:
 #고성지역은 강원도와 경남 고성을 구분해준다.
election_result.loc[125, 'ID'] = '고성(강원)'
election_result.loc[233, 'ID'] = '고성(경남)'

election_result[election_result['시군'] == '고성군']

In [ ]:
election_result[election_result['광역시도'] == '경상남도']

In [ ]:
election_result.loc[228, 'ID'] = '창원 합포'
election_result.loc[229, 'ID'] = '창원 회원'

In [ ]:
# 부천시 데이터를 3으로 나눈다.데이터 삽입시 index번호를 확인한다.

ahn_tmp = election_result.loc[85, 'ahn']/3
hong_tmp = election_result.loc[85, 'hong']/3
moon_tmp = election_result.loc[85, 'moon']/3
pop_tmp = election_result.loc[85, 'pop']/3

rate_moon_tmp = election_result.loc[85, 'rate_moon']
rate_hong_tmp = election_result.loc[85, 'rate_hong']
rate_ahn_tmp = election_result.loc[85, 'rate_ahn']

election_result.loc[250] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp, 
                           '경기도', '부천시', '부천 소사', 
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]
election_result.loc[251] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp, 
                           '경기도', '부천시', '부천 오정', 
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]
election_result.loc[252] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp, 
                           '경기도', '부천시', '부천 원미', 
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]

In [ ]:
election_result.drop([85], inplace=True)
election_result[election_result['시군'] == '부천시']

In [ ]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

In [ ]:
final_elect_data = pd.merge(election_result, draw_korea, how='left', on=['ID'])
final_elect_data.head()

In [ ]:
# 마지막으로 득표율 차이를 계산해준다.
# (문 후보와 홍 후보의 득표율 차이) -> moon_vs_hong
# (문 후보와 안 후보의 득표율 차이) -> moon_vs_ahn
# (안 후보와 홍 후보의 득표율 차이) -> ahn_vs_hong

final_elect_data['moon_vs_hong'] = final_elect_data['rate_moon'] - \
                                                                final_elect_data['rate_hong']
final_elect_data['moon_vs_ahn'] = final_elect_data['rate_moon'] - \
                                                                final_elect_data['rate_ahn']
final_elect_data['ahn_vs_hong'] = final_elect_data['rate_ahn'] - \
                                                                final_elect_data['rate_hong']
final_elect_data.head()

# 시각화-좌표

In [ ]:
# 경계선을 그리는 라인이다.
BORDER_LINES = [
    [(5, 1), (5,2), (7,2), (7,3), (11,3), (11,0)], # 인천
    [(5,4), (5,5), (2,5), (2,7), (4,7), (4,9), (7,9), 
     (7,7), (9,7), (9,5), (10,5), (10,4), (5,4)], # 서울
    [(1,7), (1,8), (3,8), (3,10), (10,10), (10,7), 
     (12,7), (12,6), (11,6), (11,5), (12, 5), (12,4), 
     (11,4), (11,3)], # 경기도
    [(8,10), (8,11), (6,11), (6,12)], # 강원도
    [(12,5), (13,5), (13,4), (14,4), (14,5), (15,5), 
     (15,4), (16,4), (16,2)], # 충청북도
    [(16,4), (17,4), (17,5), (16,5), (16,6), (19,6), 
     (19,5), (20,5), (20,4), (21,4), (21,3), (19,3), (19,1)], # 전라북도
    [(13,5), (13,6), (16,6)], # 대전시
    [(13,5), (14,5)], #세종시
    [(21,2), (21,3), (22,3), (22,4), (24,4), (24,2), (21,2)], #광주
    [(20,5), (21,5), (21,6), (23,6)], #전라남도
    [(10,8), (12,8), (12,9), (14,9), (14,8), (16,8), (16,6)], #충청북도
    [(14,9), (14,11), (14,12), (13,12), (13,13)], #경상북도
    [(15,8), (17,8), (17,10), (16,10), (16,11), (14,11)], #대구
    [(17,9), (18,9), (18,8), (19,8), (19,9), (20,9), (20,10), (21,10)], #부산
    [(16,11), (16,13)], #울산
    [(27,5), (27,6), (25,6)],
]

In [ ]:
blockedMap=final_elect_data
targetData='moon_vs_hong'
print(np.abs(min(blockedMap[targetData])))
print(np.abs(max(blockedMap[targetData])))
max([ np.abs(min(blockedMap[targetData])), np.abs(max(blockedMap[targetData]))])

In [ ]:
blockedMap.shape

In [ ]:
def drawKorea(targetData, blockedMap, cmapname):
    gamma = 0.75
    whitelabelmin = 20.
    datalabel = targetData

    tmp_max = max([ np.abs(min(blockedMap[targetData])), np.abs(max(blockedMap[targetData]))])
    vmin, vmax = -tmp_max, tmp_max

    mapdata = blockedMap.pivot_table(index='y', columns='x', values=targetData)
    masked_mapdata = np.ma.masked_where(np.isnan(mapdata), mapdata)
    
    plt.figure(figsize=(9, 11))
    plt.pcolor(masked_mapdata, vmin=vmin, vmax=vmax, cmap=cmapname, edgecolor='#aaaaaa', linewidth=0.5)

    # 지역 이름 표시
    for idx, row in blockedMap.iterrows():
        # 광역시는 구 이름이 겹치는 경우가 많아서 시단위 이름도 같이 표시한다. 
        #(중구, 서구)
        if len(row['ID'].split())==2:
            dispname = '{}\n{}'.format(row['ID'].split()[0], row['ID'].split()[1])
        elif row['ID'][:2]=='고성':
            dispname = '고성'
        else:
            dispname = row['ID']

        # 서대문구, 서귀포시 같이 이름이 3자 이상인 경우에 작은 글자로 표시한다.
        if len(dispname.splitlines()[-1]) >= 3:
            fontsize, linespacing = 10.0, 1.1
        else:
            fontsize, linespacing = 11, 1.

        annocolor = 'white' if np.abs(row[targetData]) > whitelabelmin else 'black'
        plt.annotate(dispname, (row['x']+0.5, row['y']+0.5), weight='bold',
                     fontsize=fontsize, ha='center', va='center', color=annocolor, linespacing=linespacing)

    # 시도 경계 그린다.
    for path in BORDER_LINES:
        ys, xs = zip(*path)
        plt.plot(xs, ys, c='black', lw=2)

    plt.gca().invert_yaxis()
    plt.axis('off')

    cb = plt.colorbar(shrink=.1, aspect=10)
    cb.set_label(datalabel)

    plt.tight_layout()
    plt.show()

In [ ]:
# (문재인 후보 vs 홍준표 후보)의 득표율 차이
drawKorea('moon_vs_hong', final_elect_data, 'RdBu')

In [ ]:
# folium를 이용해 결과를 비교 해본다.

import folium
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [133]:

pop_folium = final_elect_data.set_index('ID')

del pop_folium['광역시도']
del pop_folium['시군']

pop_folium.head()

,pop,moon,hong,ahn,rate_moon,rate_hong,rate_ahn,y,x,moon_vs_hong,moon_vs_ahn,ahn_vs_hong
ID,,,,,,,,,,,,
서울 종로,102566.0,42512.0,22325,22313,41.448433,21.766472,21.754773,4,6,19.681961,19.693661,-0.011700
서울 중구,82852.0,34062.0,17901,19372,41.111862,21.605996,23.381451,5,6,19.505866,17.730411,1.775455
서울 용산,148157.0,58081.0,35230,32109,39.202333,23.778829,21.672280,6,6,15.423503,17.530053,-2.106549
서울 성동,203175.0,86686.0,40566,45674,42.665682,19.966039,22.480128,5,7,22.699643,20.185554,2.514089
서울 광진,240030.0,105512.0,46368,52824,43.957839,19.317585,22.007249,6,7,24.640253,21.950590,2.689664


In [140]:

geo_path = '../testData/skorea_municipalities_geo_simple2.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

map = folium.Map(location=[36.2002, 127.054], zoom_start=6)
map.choropleth(geo_data = geo_str,
               data = pop_folium['moon_vs_hong'],
               columns = [pop_folium.index, pop_folium['moon_vs_hong']],
               fill_color = 'PuBu', #PuRd, YlGnBu
               key_on = 'feature.properties.name')